In [1]:
import pandas as pd
df = pd.read_csv("data.csv")

C:\Users\Rasengan\AppData\Local\Temp\ipykernel_6980\3422652871.py:2: DtypeWarning: Columns (9,11,12,13,14,15,16,17,18,19,26,27,36,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data.csv")


In [6]:
id_columns = [
    "FOD_ID",
    "FPA_ID",
    "NWCG_REPORTING_UNIT_ID",
    "SOURCE_REPORTING_UNIT",
    "LOCAL_FIRE_REPORT_ID",
    "LOCAL_INCIDENT_ID",
    "FIRE_CODE",
    "ICS_209_PLUS_INCIDENT_JOIN_ID",
    "ICS_209_PLUS_COMPLEX_JOIN_ID",
    "MTBS_ID",
    "FIPS_CODE",
    "MTBS_FIRE_NAME",
    "COMPLEX_NAME",
    "NWCG_CAUSE_AGE_CATEGORY",
    "Shape"
]
df_cleaned = df.drop(columns=id_columns)
df_cleaned.shape

(2303566, 24)

In [7]:
# exclude_states = ['AK', 'HI', 'PR', 'VI', 'GU', 'MP', 'AS']
exclude_states = ['PR']
df_cleaned = df_cleaned[~df_cleaned['STATE'].isin(exclude_states)]

df_cleaned.shape

(2281364, 24)

In [8]:
# df_cleaned = df_cleaned[df_cleaned["FIRE_YEAR"] >= 2000]
state_fire_counts = df_cleaned['STATE'].value_counts(normalize=True)
total_sample_size = 200000
state_sample_sizes = (state_fire_counts * total_sample_size).round().astype(int)
df_sampled = df_cleaned.groupby('STATE').apply(
    lambda x: x.sample(n=min(len(x), state_sample_sizes[x.name]), random_state=42)
).reset_index(drop=True)
df_sampled.shape

C:\Users\Rasengan\AppData\Local\Temp\ipykernel_6980\3403554508.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df_cleaned.groupby('STATE').apply(


(199997, 24)

In [9]:
df_map = df_sampled[df_sampled["FIRE_SIZE"] > 0]

columns_to_keep = [
    "FIRE_NAME",
    "FIRE_SIZE",
    "DISCOVERY_DATE",
    "LATITUDE",
    "LONGITUDE",
    "NWCG_CAUSE_CLASSIFICATION",
    "STATE",
    "FIRE_YEAR",
]

df_map = df_map[columns_to_keep].copy()

df_map.to_json(f'wildfires_{total_sample_size}_for_map.json', orient="records", mode="w")